# Import

In [38]:
import sys

import pandas as pd
import numpy as np

from IPython.display import display, HTML

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from dataclasses import dataclass

from scipy.stats import ttest_ind as t_test

In [39]:
def t_test_races_at_9_mph(dataframe:pd.DataFrame, speed_range:tuple) -> pd.DataFrame:
    min_speed = speed_range[0]
    max_speed = speed_range[1]
    
    for race in race_set:
        speed_eq_9 = dataframe[(dataframe['Speed over Limit'] <= max_speed) 
                                & (dataframe['Speed over Limit'] >= min_speed)
                                & (dataframe['Race']==race)]['Speed over Limit']==9
        
        proportion_at_9 = speed_eq_9.mean()
        std_of_eq_9 = speed_eq_9.std()
        
        

In [40]:
def t_test_races_at_9_mph(dataframe:pd.DataFrame, speed_range:tuple) -> pd.DataFrame:
    min_speed = speed_range[0]
    max_speed = speed_range[1]
    
    for race in race_set:
        speed_eq_9 = dataframe[(dataframe['Speed over Limit'] <= max_speed) 
                                & (dataframe['Speed over Limit'] >= min_speed)
                                & (dataframe['Race']==race)]['Speed over Limit']==9
        
        proportion_at_9 = speed_eq_9.mean()
        std_of_eq_9 = speed_eq_9.std()
        
        

# One row per stop

In [41]:
one_row_per_stop_df = pd.read_csv(filepath_or_buffer=r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/Policing Thesis/Modified Dataset - 2021 - One Row per Stop.csv",
                 index_col='Stop ID')

C:\Users\mikha\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning:

Columns (18,19,20,21,22,23,32,44,45,46,47,60) have mixed types.Specify dtype option on import or set low_memory=False.



In [42]:
one_row_per_stop_df['Speed over Limit'] = one_row_per_stop_df['Recorded Speed - Sparse Filled'] - one_row_per_stop_df['Speed Limit - Sparse Filled']

In [43]:
one_row_per_stop_df = one_row_per_stop_df[one_row_per_stop_df['Speed over Limit'] < 80]

In [44]:
race_set = ['WHITE',
'ASIAN',
'HISPANIC',
'OTHER',
'BLACK']

In [45]:
speeding_one_row_per_stop_df = one_row_per_stop_df[one_row_per_stop_df['Recorded Speed'].notnull()]

In [52]:
def plot_speeding_hist(dataframe:pd.DataFrame, histfunc=None, xrange=tuple):

    x_lower_lim = xrange[0]
    x_upper_lim = xrange[1]

    if histfunc is None:
        histfunc = 'probability'

    hists_list = [go.Histogram(
        histnorm=histfunc,
        x=dataframe[(dataframe['Speed over Limit'] >= x_lower_lim) 
                    & (dataframe['Speed over Limit'] <= x_upper_lim) 
                    & (dataframe['Race']==race)]['Speed over Limit'],
        name=race,
        opacity=.85)
                for race in race_set]

    fig = go.Figure()
    for hist in hists_list:
        fig.add_trace(hist)
        
    fig.update_xaxes(range=[x_lower_lim-1, x_upper_lim+1],
                     title={'text':'Speed over Limit<br><sup>(MPH)<sup>'})
 
    fig.update_yaxes(range=[0, .7],
                 tickformat=',.0%')
    
    height=500
    fig.update_layout(autosize=False, height=height, width=height*1.81803398875,
                      margin={'t':40, 'b':0},
                      xaxis = dict(
                        dtick = 1
    ))
    
    # fig.update_layout(title='Recorded Speeding by Race',
    #                   title_x=0.5)

    # fig.show()
    
    return fig


In [54]:
one_row_speeding_lot = plot_speeding_hist(one_row_per_stop_df, xrange=(9,19))
one_row_speeding_lot.update_layout(title='Recorded Speeding by Group',title_x=0.5) #<br><sup> Number of Drivers<sup>
one_row_speeding_lot.update_yaxes(title={'text':"Percent of Own Group's Stops"})
one_row_speeding_lot.write_image('Recorded Speeding by Group - Stops.png', scale=5)
one_row_speeding_lot.write_html('Recorded Speeding by Group - Stops.html')
one_row_speeding_lot.show()

# All rows

In [29]:
df = pd.read_csv(filepath_or_buffer=r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/Policing Thesis/Modified Dataset - 2021.csv")

C:\Users\mikha\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning:

Columns (18,19,20,21,22,23,32,44,45,46,47,60) have mixed types.Specify dtype option on import or set low_memory=False.



In [30]:
df.set_index('Stop ID', inplace=True)

In [31]:
df['Speed over Limit'] = df['Recorded Speed - Sparse Filled'] - df['Speed Limit - Sparse Filled']

In [146]:
all_rows_speeding_plot = plot_speeding_hist(df, 'probability density')
all_rows_speeding_plot.update_yaxes(title={'text':"Own Race's <b>Writeups<b>"})
all_rows_speeding_plot.update_layout(title='Recorded Speeding by Race',title_x=0.5) # <br><sup> Number of Citations<sup>
all_rows_speeding_plot.write_image('Recorded Speeding by Race - Writeups.png', scale=5)
all_rows_speeding_plot.show()

In [33]:
plot_speeding_hist(df, 'probability')

# Probable Cause searches by mph

In [39]:
p_cause_race_speed = df.groupby(by=['Speed over Limit', 'Race'])['Probable Cause - Sparse Filled'].agg(['mean','count']).reset_index()

In [40]:
p_cause_race_speed

,Speed over Limit,Race,mean,count
0,0.0,ASIAN,0.000000,8
1,0.0,BLACK,0.029412,34
2,0.0,HISPANIC,0.000000,86
3,0.0,OTHER,0.000000,4
4,0.0,WHITE,0.018182,110
...,...,...,...,...
368,3900.0,WHITE,0.000000,2
369,4217.0,BLACK,0.000000,1
370,4900.0,BLACK,0.000000,1
371,4900.0,WHITE,0.000000,1


In [45]:
def plot_p_cause_speeding_hist(dataframe:pd.DataFrame, histfunc=None):

    x_upper_lim = 30

    if histfunc is None:
        histfunc = 'percent'

    #ADD NUM STOPS HOVER DATA
    hists_list = [go.Bar(
        x=dataframe[(dataframe['Speed over Limit'] < x_upper_lim) & (dataframe['Race']==race) & (dataframe['count']>10)]['Speed over Limit'],
        y=dataframe[(dataframe['Speed over Limit'] < x_upper_lim) & (dataframe['Race']==race) & (dataframe['count']>10)]['mean'],
        name=race)
                for race in race_set]

    fig = go.Figure()
    for hist in hists_list:
        fig.add_trace(hist)
        
    fig.update_xaxes(range=[5, x_upper_lim])

    fig.show()
    

plot_p_cause_speeding_hist(p_cause_race_speed)

# Citations per 9 mph stop, by race

In [35]:
# df[df['Race']=='BLACK']


# T-tests

In [ ]:
@dataclass
class race_stats_for_testing:
    name: str
    proportion: float
    std: float
    
    def t_test(self, other):
        

In [ ]:
def t_test_races_at_9_mph(dataframe:pd.DataFrame, speed_range:tuple) -> pd.DataFrame:
    min_speed = speed_range[0]
    max_speed = speed_range[1]
    
    
    ########### Create 
    dataclass_storage_dict = dict()
    
    for race in race_set:
        speed_eq_9 = dataframe[(dataframe['Speed over Limit'] <= max_speed) 
                                & (dataframe['Speed over Limit'] >= min_speed)
                                & (dataframe['Race']==race)]['Speed over Limit']==9
        
        proportion = speed_eq_9.mean()
        std = speed_eq_9.std()
        
        dataclass_storage_dict[race] = race_stats_for_testing(name=race, 
                                                              proportion=proportion,
                                                              std=std)
        
    

In [88]:
def t_test_races_at_9_mph(dataframe:pd.DataFrame, speed_range:tuple) -> pd.DataFrame:
    min_speed = speed_range[0]
    max_speed = speed_range[1]
    
    race_equals_9_dict = dict()
    
    for race in race_set:
        race_equals_9_dict[race] = (dataframe[(dataframe['Speed over Limit'].notnull()) #first cond probably isn't necessary
                                             & (dataframe['Speed over Limit'] <= max_speed) 
                                             & (dataframe['Speed over Limit'] >= min_speed)
                                             & (dataframe['Race']==race)]['Speed over Limit']==9).astype(int)
    
    t_test_results_dict = {race:{} for race in race_set}
    
    for race1 in race_set:
        

        t_test_results_dict[race1] = [t_test(a=race_equals_9_dict[race1], 
                                             b=race_equals_9_dict[race2], 
                                             equal_var=False).pvalue
                                      for race2 in race_set]
        
    p_values_df = pd.DataFrame.from_dict(t_test_results_dict)
    p_values_df.index = race_set
    
    return p_values_df
    

In [98]:
t_test_races_at_9_mph(one_row_per_stop_df, speed_range=(0,30))

,WHITE,ASIAN,HISPANIC,OTHER,BLACK
WHITE,1.000,0.003,0.000,0.000,0.000
ASIAN,0.003,1.000,0.000,0.000,0.000
HISPANIC,0.000,0.000,1.000,0.731,0.000
OTHER,0.000,0.000,0.731,1.000,0.000
BLACK,0.000,0.000,0.000,0.000,1.000


In [99]:
t_test_races_at_9_mph(df, speed_range=(0,30))

,WHITE,ASIAN,HISPANIC,OTHER,BLACK
WHITE,1.000,0.018,0.000,0.000,0.000
ASIAN,0.018,1.000,0.000,0.000,0.000
HISPANIC,0.000,0.000,1.000,0.006,0.000
OTHER,0.000,0.000,0.006,1.000,0.000
BLACK,0.000,0.000,0.000,0.000,1.000


In [96]:
pd.set_option('display.float_format', lambda x: f'{x:,.3f}')

In [97]:
t_tests

,WHITE,ASIAN,HISPANIC,OTHER,BLACK
WHITE,1.000,0.003,0.000,0.000,0.000
ASIAN,0.003,1.000,0.000,0.000,0.000
HISPANIC,0.000,0.000,1.000,0.731,0.000
OTHER,0.000,0.000,0.731,1.000,0.000
BLACK,0.000,0.000,0.000,0.000,1.000


In [89]:
pd.DataFrame.from_dict(t_tests)

,WHITE,ASIAN,HISPANIC,OTHER,BLACK
0,1.000000e+00,2.683771e-03,9.473523e-30,5.449512e-18,2.837569e-98
1,2.683771e-03,1.000000e+00,2.059795e-06,1.388239e-05,1.314246e-21
2,9.473523e-30,2.059795e-06,1.000000e+00,7.311118e-01,5.819471e-09
3,5.449512e-18,1.388239e-05,7.311118e-01,1.000000e+00,4.530088e-05
4,2.837569e-98,1.314246e-21,5.819471e-09,4.530088e-05,1.000000e+00
